## Test Injection of Data

- This note book try to import parquet file into postgreSQL database
- Remember to implement the click part to take in command line parameters
- When done it will be converted to script using nbconvert

In [2]:
import pandas as pd
#import click
from sqlalchemy import create_engine
from tqdm.auto import tqdm
import pyarrow.parquet as pq

parquet_path = "./green_tripdata_2025-11.parquet"

pg_user = 'postgres'
pg_pass = 'postgres'
pg_host = 'localhost'
pg_port = '5433'
pg_db = 'ny_taxi'

target_table = 'green_taxi_trips'

engine = create_engine(f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')

first = True
preview_df = None

parquet_file = pq.ParquetFile(parquet_path)

for batch in tqdm(parquet_file.iter_batches(batch_size=100000)):
    df_chunk = batch.to_pandas()

    if first:
        df_chunk.head(0).to_sql(
            name=target_table,
            con=engine,
            if_exists='replace'
        )
        preview_df = df_chunk.head()
        first = False

    df_chunk.to_sql(
        name=target_table,
        con=engine,
        if_exists='append'
    )

if preview_df is not None:
    print(preview_df)

0it [00:00, ?it/s]

   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2025-11-01 00:34:48   2025-11-01 00:41:39                  N   
1         2  2025-11-01 00:18:52   2025-11-01 00:24:27                  N   
2         2  2025-11-01 01:03:14   2025-11-01 01:15:24                  N   
3         2  2025-11-01 00:10:57   2025-11-01 00:24:53                  N   
4         1  2025-11-01 00:03:48   2025-11-01 00:19:38                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0         1.0            74            42              1.0           0.74   
1         1.0            74            42              2.0           0.95   
2         1.0            83           160              1.0           2.19   
3         1.0           166           127              1.0           5.44   
4         1.0           166           262              1.0           3.20   

   fare_amount  ...  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0    